This example requires you to get an OPENAI_API_KEY. 


In [ ]:
# Jupyter-friendly version of ZRetriever usage

import nest_asyncio
import asyncio
from zmongo_toolbag.zmongo import ZMongo
from zmongo_toolbag.zmongo_retriever import ZRetriever

# Allow nested event loops for Jupyter compatibility
nest_asyncio.apply()

# Define the async logic directly
async def run_zretriever_demo():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)

    # Replace with real collection name and document IDs
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')

    for idx, doc_set in enumerate(documents):
        print(f"\nDocument Set {idx + 1}:")
        for doc in doc_set:
            print(f"Metadata: {doc.metadata}")
            print(f"Content: {doc.page_content[:200]}...\n")

# Run the async function in Jupyter
await run_zretriever_demo()


Set your variables

Use ZMongoRetriever to split the text from the page_content_field into LangChain Documents

If the output from above == [] then you have a problem with your variables.

Use LlamaCPP to summarize the chunked output.

In [ ]:
# Jupyter cell: Combine ZRetriever with LlamaModel to query MongoDB and generate LLM completions

import os
import nest_asyncio
import asyncio
from dotenv import load_dotenv
from zmongo_toolbag.zmongo import ZMongo
from zmongo_toolbag.zmongo_retriever import ZRetriever
from llama_cpp import Llama

# Load environment variables and allow nested asyncio
load_dotenv('/home/overlordx/resources/.env')
nest_asyncio.apply()

class LlamaModel:
    def __init__(self):
        self.model_path = os.getenv(
            "GGUF_MODEL_PATH",

        )
        self.n_ctx = int(os.getenv("N_CTX", 512))
        self.n_batch = int(os.getenv("N_BATCH", 126))
        self.llm = None
        self.load_model()

    def load_model(self):
        print("Loading model...")
        self.llm = Llama(
            model_path=self.model_path,
            n_ctx=self.n_ctx,
            n_batch=self.n_batch
        )
        print("Model loaded successfully.")

    def generate_prompt_from_template(self, user_input: str) -> str:
        return (
            "<|im_start|>system\n"
            "You are a helpful chatbot.<|im_end|>\n"
            "<|im_start|>user\n"
            f"{user_input}<|im_end|>"
        )

    def generate_text(self, prompt: str, max_tokens: int = 256, temperature: float = 0.1, top_p: float = 0.5, echo: bool = False, stop: list = None) -> str:
        stop = stop or ["#"]
        output = self.llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            echo=echo,
            stop=stop,
        )
        return output["choices"][0]["text"].strip()

# Main async function to retrieve and summarize using Llama
async def run_llama_zretriever_pipeline():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')

    llama_model = LlamaModel()

    for idx, doc_set in enumerate(documents):
        combined_text = "\n\n".join(doc.page_content for doc in doc_set)
        print(f"\nDocument Set {idx + 1} (retrieved {len(doc_set)} chunks):")
        prompt = llama_model.generate_prompt_from_template(
            f"Summarize the following legal content for a law student:\n{combined_text[:1000]}"
        )
        summary = llama_model.generate_text(prompt, max_tokens=512)
        print("\nSummary:")
        print(summary)

# Run in notebook
await run_llama_zretriever_pipeline()


In [ ]:
# Jupyter cell: Combine ZRetriever with OpenAIModel to query MongoDB and generate completions

import os
import nest_asyncio
import asyncio
from bson import ObjectId
from dotenv import load_dotenv
from zmongo_toolbag.zmongo import ZMongo
from zmongo_toolbag.zmongo_retriever import ZRetriever
import openai

# Load environment variables and allow nested asyncio
load_dotenv('/home/overlordx/resources/.env')
nest_asyncio.apply()
openai.api_key = os.getenv("OPENAI_API_KEY_APP")

class OpenAIModel:
    def __init__(self):
        self.model = os.getenv("OPENAI_TEXT_MODEL", "gpt-3.5-turbo-instruct")
        self.max_tokens = int(os.getenv("DEFAULT_MAX_TOKENS", 256))
        self.temperature = float(os.getenv("DEFAULT_TEMPERATURE", 0.7))
        self.top_p = float(os.getenv("DEFAULT_TOP_P", 0.95))

    async def _call_openai(self, prompt: str, max_tokens=None, temperature=None, top_p=None, stop=None, echo=False) -> str:
        try:
            response = await asyncio.to_thread(openai.completions.create,
                model=self.model,
                prompt=prompt,
                max_tokens=max_tokens or self.max_tokens,
                temperature=temperature or self.temperature,
                top_p=top_p or self.top_p,
                stop=stop,
                echo=echo,
            )
            return response.choices[0].text.strip()
        except Exception as e:
            return f"[OpenAI Error] {e}"

    async def summarize_text(self, text: str) -> str:
        prompt = f"Summarize the following for a legal researcher:\n\n{text}\n\nSummary:"
        return await self._call_openai(prompt, max_tokens=200)

# Main async function to retrieve and summarize using OpenAI
async def run_openai_zretriever_pipeline():
    repo = ZMongo()
    retriever = ZRetriever(repository=repo, max_tokens_per_set=4096, chunk_size=512)
    collection_name = 'documents'
    document_ids = ['67e5ba645f74ae46ad39929d', '67ef0bd71a349c7c108331a6']

    documents = await retriever.invoke(collection=collection_name, object_ids=document_ids, page_content_key='text')
    openai_model = OpenAIModel()

    for idx, doc_set in enumerate(documents):
        combined_text = "\n\n".join(doc.page_content for doc in doc_set)
        print(f"\nDocument Set {idx + 1} (retrieved {len(doc_set)} chunks):")
        summary = await openai_model.summarize_text(combined_text[:1000])
        print("\nSummary:")
        print(summary)

# Run in notebook
await run_openai_zretriever_pipeline()
